In [1]:
import pandas as pd
# Geopandas is a geo dataframe
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Load combined data set

In [2]:
all = pd.read_csv("/home/kim/data/bedmap/bedmap123.csv")
# loading data takes 2 min, 67.7 M data points

In [ ]:
all.head()

In [ ]:
all.shape

# Preprocess

## Check bounds

In [ ]:
all = all.drop(all.loc[(all['x'] > 3333000) | (all['x'] < -3333000) | (all['y'] > 3333000) | (all['y'] < -3333000)].index)

all.shape

## Check b,s,t, replaced with 9999 instead of -9999

### b == 9999

In [ ]:
# 6 values from BEDMAP1_1966-2000_AIR_BM1.csv but with s > 10000 so I am removing these
all.loc[all["b"] == 9999]

In [ ]:
all = all.drop(all.loc[all["b"] == 9999].index)

### s == 9999

In [ ]:
# 2 values but also something clearly wrong
all.loc[all["s"] == 9999]

In [ ]:
all = all.drop(all.loc[all["s"] == 9999].index)

### t == 9999

In [ ]:
all.loc[all["t"] == 9999]

### t, b given, s missing

# Surface missing, t and b given: can't check really
all.loc[(all['s'] == -9999) & (all['t'] != -9999) & (all['b'] != -9999)]

## b, s given, t missing

# Thickness missing, b & s given
# Check if b > s: 92 k
# Bedmap paper states imputation
bed_surface_threshold = 0.0
all.loc[(all['s'] != -9999) & (all['t'] == -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)]

In [ ]:
# remove
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] == -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)].index)

### traingle:

In [ ]:
# Inspect values where all 3 fields are given and b > s
# > 2 M 
all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (all['s'] < all['b'])]

In [ ]:
# deleting an extra 100k because of inconsistency
traingle_threshold = 5.0
all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (np.abs(all['s'] - all['t'] - all['b']) > traingle_threshold)]

In [ ]:
# Triangle check for all complete values
traingle_threshold = 5.0
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (np.abs(all['s'] - all['t'] - all['b']) > traingle_threshold)].index)

In [ ]:
all.shape

In [ ]:
# Repeat with stricter threshold for b > s
# bed elevation > surface
bed_surface_threshold = 0.0
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)].index)

# Very high bedrock

In [ ]:
all.shape

In [ ]:
all.loc[(all['b'] > 3600)]["source"].value_counts()

In [ ]:
all.loc[(all['b'] > 3600) & (all["source"] == "BEDMAP1_1966-2000_AIR_BM1.csv")]

In [ ]:
# appears to be an error
all.loc[(all['b'] > 3600) & (all["source"] == "NASA_2012_ICEBRIDGE_AIR_BM2.csv")]

In [ ]:
# way too high
all.loc[(all['b'] > 3600) & (all["source"] == "UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv")]

In [ ]:
# way too high
all.loc[(all['b'] > 3600) & (all["source"] == "NASA_2010_ICEBRIDGE_AIR_BM2.csv")]

In [ ]:
all = all.drop(all.loc[(all['b'] > 3600)].index)

### Very high surface

In [ ]:
# plausible
all.loc[(all['s'] > 4200)]

In [ ]:
# ice is thick where the bed is very low
all.loc[(all['t'] > 4700)]

# Export cleaned data

In [ ]:
all.to_csv("/home/kim/data/bedmap/bedmap123_preprocessed.csv")